# Example of performing SHAP analysis

## Import packages

In [1]:
import numpy as np
import pandas as pd
import shap
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import time

In [2]:
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


## Define a helper function to process the SHAP values from the explainer

In [3]:
def process_shap(X, shap, rows=18, cols=12):
    two_d_list = [[[] for j in range(cols)] for i in range(rows)]
    for i in range(X.shape[0]):
        seq = X[i]
        for j in range(len(seq)):
            two_d_list[seq[j]][j].append(shap[i][j])
    return two_d_list

## Load the trained score predictor

In [4]:
model = keras.models.load_model('../score_predictors/pe/trained_model/')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## Prepare the background data and implement the explainer

In [5]:
df = pd.read_csv('../data/sequence_score_data_pe_processed.csv')
sequences = df['Short Sequence'].to_list()
scores = df['NET_E'].to_list()

amino_acid_alphabet = ['A', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 
                       'M', 'N', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

encoder = LabelEncoder()
encoder.fit(amino_acid_alphabet)

encoded_sequences = [encoder.transform(list(seq)) for seq in sequences]
X = pad_sequences(encoded_sequences)

y = np.array(scores)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [6]:
np.random.seed(42)
rand_ind = np.random.choice(X_train.shape[0], 1000)
explainer = shap.DeepExplainer(model, [X_train[rand_ind]])

keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


## Load the dataset to be analyzed

In [7]:
# A small dataset containing 500 peptide sequences and their predicted scores are used for demonstration purpose
df_s = pd.read_csv('../data/sample_data.csv')

In [8]:
sequences_s = df_s['Sequences'].to_list()
scores_s = df_s['Scores'].to_list()

encoded_sequences_s = [encoder.transform(list(seq)) for seq in sequences_s]
X_s = pad_sequences(encoded_sequences_s)
y_s = np.array(scores_s)

## Compute the SHAP values for the amino acids in the peptides

In [9]:
start = time.time()
shap_values_s = explainer.shap_values(X_s, check_additivity=False)
print("Time elapsed: ", time.time()-start)

Time elapsed:  75.10459327697754


In [10]:
y_s_2d  = np.repeat(y_s.reshape(-1, 1), 12, axis=1)
two_d_list = process_shap(X_s, np.squeeze(np.asarray(shap_values_s)))
two_d_list_score = process_shap(X_s, y_s_2d)

rows, cols = len(two_d_list), len(two_d_list[0])
data = []

for i in range(rows):
    for j in range(cols):
        for val, score in zip(two_d_list[i][j], two_d_list_score[i][j]):
            data.append([i, j, val, score])

df_shap = pd.DataFrame(data, columns=['Amino Acid Number', 'Position', 'SHAP Value', 'PepBD Score'])
print(df_shap)

      Amino Acid Number  Position  SHAP Value  PepBD Score
0                     0         0   -0.204441   -51.974070
1                     0         0    0.351497   -53.508335
2                     0         0    0.407673   -49.370213
3                     0         0    0.118165   -53.585773
4                     0         0    0.267974   -55.235180
...                 ...       ...         ...          ...
5995                 17        11   -0.486788   -47.682460
5996                 17        11   -0.291190   -43.469960
5997                 17        11   -0.809531   -48.178486
5998                 17        11   -0.793417   -47.565662
5999                 17        11   -0.273988   -48.618380

[6000 rows x 4 columns]


In [11]:
# df_shap.to_csv('./shap_test.csv', index=False)